In [ ]:
import csv
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup

In [ ]:
counter = 0
missing_movies = []
# These row names are weird, because some movies have a ':' in their title.
row_names = ['movie_id', 'blank', 'movie_title','blank2', 'genres', 'genres2']

with open('movies.txt', 'r', encoding = "ISO-8859-1") as f:
    reader = csv.DictReader(f, fieldnames=row_names, delimiter=':')
    for row in reader:
        movie_id = row['movie_id']
        movie_title = row['movie_title']
        
        genres = list(row['genres'].split("|"))
        if genres == ['']:
          genres = list(row['genres2'].split("|"))
        
        domain = 'http://www.imdb.com'
        search_url = domain + '/find?q=' + urllib.parse.quote_plus(movie_title)
        with urllib.request.urlopen(search_url) as response:
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
            # Get url of 1st search result
            try:
                title = soup.find('table', class_='findList').tr.a['href']
                movie_url = domain + title
                with open('movie_url.csv', 'a', newline='') as out_csv:
                    writer = csv.writer(out_csv, delimiter=',')
                    writer.writerow([movie_id, movie_url, genres])
            # Ignore cases where search returns no results
            # Save them, for gathering them manually later
            except AttributeError:
                print("Movie not found: " + movie_id)
                missing_movies1.append(movie_id)
                counter += 1
                
print("Missing movies:", counter)
print("Done")

In [ ]:
counter = 0
missing_posters = []
row_names = ['movie_id', 'movie_url', 'genres']

with open('movie_url.csv', 'r', newline='') as in_csv:
    reader = csv.DictReader(in_csv, fieldnames=row_names, delimiter=',')
    for row in reader:
        movie_id = row['movie_id']
        movie_url = row['movie_url']
        
        # Filter out the square brackets, space, and single and double quotes
        my_table = row['genres'].maketrans('','','[] \'"')
        genres = list(row['genres'].translate(my_table).split(','))  
        
        domain = 'http://www.imdb.com'
        with urllib.request.urlopen(movie_url) as response:
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
            # Get url of poster image
            try:
                image_url = soup.find('div', class_='poster').a.img['src']
                extension = '.jpg'
                image_url = ''.join(image_url.partition('_')[0]) + extension
                
#                 # By doing this you can split the images in folders per genre.
#                 for genre in genres:
#                     filename = 'movies_data/' + genre + '/' + movie_id + extension
#                     with urllib.request.urlopen(image_url) as response:
#                         with open(filename, 'wb') as out_image:
#                             out_image.write(response.read())
                
                # By doing this, you put all the images in the same folder
                filename = 'movies_data_split/' + movie_id + extension
                with urllib.request.urlopen(image_url) as response:
                    with open(filename, 'wb') as out_image:
                        out_image.write(response.read())
                        
                with open('movie_poster.csv', 'a', newline='') as out_csv:
                    writer = csv.writer(out_csv, delimiter=',')
                    writer.writerow([movie_id, image_url])
                    
            # Ignore cases where no poster image is present
            # Save them, for gathering them manually later
            except AttributeError:
                print("Missing poster: " + movie_id)
                missing_posters.append(movie_id)
                counter += 1

print("Missing posters:", counter)
print("Done")